In [3]:
import torch

torch_version = str(torch.__version__)
scatter_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
sparse_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
!pip install torch-scatter -f $scatter_src
!pip install torch-sparse -f $sparse_src
!pip install torch-geometric

In [4]:
import torch
from torch_geometric.nn import SAGEConv
from torch_geometric.transforms import NormalizeFeatures

In [5]:
class GraphSAGE(torch.nn.Module):
  def __init__(self, in_channels, hidden_channels, out_channels):
    super(GraphSAGE, self).__init__()
    self.conv1 = SAGEConv(in_channels, hidden_channels)
    self.conv2 = SAGEConv(hidden_channels, out_channels)

  def forward(self, x, edge_index):
    x = self.conv1(x, edge_index)
    x = torch.relu(x)
    x = self.conv2(x, edge_index)
    return x

In [ ]:
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='/tmp/Cora', name = 'Cora', transform=NormalizeFeatures())
data = dataset[0]

In [7]:
x = dataset.num_node_features
y = dataset.num_classes

model = GraphSAGE(x, 16, y)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [8]:
model

GraphSAGE(
  (conv1): SAGEConv(1433, 16, aggr=mean)
  (conv2): SAGEConv(16, 7, aggr=mean)
)

In [36]:
model.train()
for epoch in range(200):
  optimizer.zero_grad()
  out = model(data.x, data.edge_index)
  loss = torch.nn.functional.cross_entropy(out[data.train_mask], data.y[data.train_mask])
  loss.backward()
  optimizer.step()

In [37]:
model.eval()
_, pred = model(data.x, data.edge_index).max(dim = 1)
correct = int(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.8980


Using Mini-Batches

In [38]:
from torch_geometric.loader import NeighborLoader, DataLoader

loader = NeighborLoader(data, input_nodes= data.train_mask, batch_size = 128, num_neighbors=[10, 5])

In [39]:
def train_N(epoch):
  model.train()
  optimizer.zero_grad()

  losses =[]
  for batch in loader:
    out = model(batch.x, batch.edge_index)
    loss = torch.nn.functional.cross_entropy(out, batch.y)
    losses.append(loss)
    loss.backward()
    optimizer.step()

  return loss

In [40]:
for epoch in range(1, 11):
  loss = train_N(epoch)
  print(f'Epoch: {epoch:03d}, Loss: {loss:.04f}')

Epoch: 001, Loss: 0.0002
Epoch: 002, Loss: 0.0002
Epoch: 003, Loss: 0.0002
Epoch: 004, Loss: 0.0002
Epoch: 005, Loss: 0.0002
Epoch: 006, Loss: 0.0002
Epoch: 007, Loss: 0.0002
Epoch: 008, Loss: 0.0002
Epoch: 009, Loss: 0.0002
Epoch: 010, Loss: 0.0002
